In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

pd.options.display.float_format = "{:,.4f}".format

In [ ]:
# Black-Scholes European-Options Gamma
def calcGammaEx(S, K, vol, T, r, q, optType, OI):
    if T == 0 or vol == 0:
        return 0

    dp = (np.log(S / K) + (r - q + 0.5 * vol**2) * T) / (vol * np.sqrt(T))
    dm = dp - vol * np.sqrt(T)

    if optType == "call":
        gamma = np.exp(-q * T) * norm.pdf(dp) / (S * vol * np.sqrt(T))
        return OI * 100 * S * S * 0.01 * gamma
    else:  # Gamma is same for calls and puts. This is just to cross-check
        gamma = K * np.exp(-r * T) * norm.pdf(dm) / (S * S * vol * np.sqrt(T))
        return OI * 100 * S * S * 0.01 * gamma


def isThirdFriday(d):
    return d.weekday() == 4 and 15 <= d.day <= 21

In [ ]:
# Inputs and Parameters
filename = "cboe_data/spx_quotedata.csv"

In [ ]:
# This assumes the CBOE file format hasn't been edited, i.e. table begins at line 4
optionsFile = open(filename)
optionsFileData = optionsFile.readlines()
optionsFile.close()

# Get SPX Spot
spotLine = optionsFileData[1]
spotPrice = float(spotLine.split(",")[1].split("Last:")[1])
fromStrike = 0.85 * spotPrice
toStrike = 1.15 * spotPrice

dateLine = optionsFileData[4].split(",")[0]
todayDate = datetime.strptime(dateLine, "%a %b %d %Y")
optionsFile.close()


# Get SPX Options Data
df = pd.read_csv(filename, sep=",", header=None, skiprows=4)
df.columns = [
    "ExpirationDate",
    "Calls",
    "CallLastSale",
    "CallNet",
    "CallBid",
    "CallAsk",
    "CallVol",
    "CallIV",
    "CallDelta",
    "CallGamma",
    "CallOpenInt",
    "StrikePrice",
    "Puts",
    "PutLastSale",
    "PutNet",
    "PutBid",
    "PutAsk",
    "PutVol",
    "PutIV",
    "PutDelta",
    "PutGamma",
    "PutOpenInt",
]
# df.loc[0]['ExpirationDate'].date().strftime('%Y-%m-%d')
df["ExpirationDate"] = pd.to_datetime(df["ExpirationDate"], format="%a %b %d %Y")
df["ExpirationDate"] = df["ExpirationDate"] + timedelta(hours=16)
df["StrikePrice"] = df["StrikePrice"].astype(float)
df["CallIV"] = df["CallIV"].astype(float)
df["PutIV"] = df["PutIV"].astype(float)
df["CallGamma"] = df["CallGamma"].astype(float)
df["PutGamma"] = df["PutGamma"].astype(float)
df["CallOpenInt"] = df["CallOpenInt"].astype(float)
df["PutOpenInt"] = df["PutOpenInt"].astype(float)


# ---=== CALCULATE SPOT GAMMA ===---
# Gamma Exposure = Unit Gamma * Open Interest * Contract Size * Spot Price
# To further convert into 'per 1% move' quantity, multiply by 1% of spotPrice
df["CallGEX"] = df["CallGamma"] * df["CallOpenInt"] * 100 * spotPrice * spotPrice * 0.01
df["PutGEX"] = (
    df["PutGamma"] * df["PutOpenInt"] * 100 * spotPrice * spotPrice * 0.01 * -1
)

df["TotalGamma"] = (df.CallGEX + df.PutGEX) / 10**9
dfAgg = df.groupby(["StrikePrice"]).sum()
strikes = dfAgg.index.values

In [ ]:
# Chart 1: Absolute Gamma Exposure
plt.grid()
plt.bar(
    strikes,
    dfAgg["TotalGamma"].to_numpy(),
    width=6,
    linewidth=0.1,
    edgecolor="k",
    label="Gamma Exposure",
)
plt.xlim([fromStrike, toStrike])
chartTitle = (
    "Total Gamma: $"
    + str("{:.2f}".format(df["TotalGamma"].sum()))
    + " Bn per 1% SPX Move"
)
plt.title(chartTitle, fontweight="bold", fontsize=15)
plt.xlabel("Strike", fontweight="bold")
plt.ylabel("Spot Gamma Exposure ($ billions/1% move)", fontweight="bold", fontsize=8)
plt.axvline(
    x=spotPrice, color="r", lw=1, label="SPX Spot: " + str("{:,.0f}".format(spotPrice))
)
plt.legend()
plt.show()

In [ ]:
# Chart 2: Absolute Gamma Exposure by Calls and Puts
plt.grid()
plt.bar(
    strikes,
    dfAgg["CallGEX"].to_numpy() / 10**9,
    width=6,
    linewidth=0.1,
    edgecolor="k",
    label="Call Gamma",
)
plt.bar(
    strikes,
    dfAgg["PutGEX"].to_numpy() / 10**9,
    width=6,
    linewidth=0.1,
    edgecolor="k",
    label="Put Gamma",
)
plt.xlim([fromStrike, toStrike])
chartTitle = (
    "Total Gamma: $"
    + str("{:.2f}".format(df["TotalGamma"].sum()))
    + " Bn per 1% SPX Move"
)
plt.title(chartTitle, fontweight="bold", fontsize=15)
plt.xlabel("Strike", fontweight="bold")
plt.ylabel("Spot Gamma Exposure ($ billions/1% move)", fontweight="bold", fontsize=8)
plt.axvline(
    x=spotPrice, color="r", lw=1, label="SPX Spot:" + str("{:,.0f}".format(spotPrice))
)
plt.legend()
plt.show()


# ---=== CALCULATE GAMMA PROFILE ===---
levels = np.linspace(fromStrike, toStrike, 60)

# For 0DTE options, I'm setting DTE = 1 day, otherwise they get excluded
df["daysTillExp"] = [
    1 / 262
    if (np.busday_count(todayDate.date(), x.date())) == 0
    else np.busday_count(todayDate.date(), x.date()) / 262
    for x in df.ExpirationDate
]

nextExpiry = df["ExpirationDate"].min()

df["IsThirdFriday"] = [isThirdFriday(x) for x in df.ExpirationDate]
thirdFridays = df.loc[df["IsThirdFriday"] == True]
nextMonthlyExp = thirdFridays["ExpirationDate"].min()

totalGamma = []
totalGammaExNext = []
totalGammaExFri = []

# For each spot level, calc gamma exposure at that point
for level in levels:
    df["callGammaEx"] = df.apply(
        lambda row: calcGammaEx(
            level,
            row["StrikePrice"],
            row["CallIV"],
            row["daysTillExp"],
            0,
            0,
            "call",
            row["CallOpenInt"],
        ),
        axis=1,
    )

    df["putGammaEx"] = df.apply(
        lambda row: calcGammaEx(
            level,
            row["StrikePrice"],
            row["PutIV"],
            row["daysTillExp"],
            0,
            0,
            "put",
            row["PutOpenInt"],
        ),
        axis=1,
    )

    totalGamma.append(df["callGammaEx"].sum() - df["putGammaEx"].sum())

    exNxt = df.loc[df["ExpirationDate"] != nextExpiry]
    totalGammaExNext.append(exNxt["callGammaEx"].sum() - exNxt["putGammaEx"].sum())

    exFri = df.loc[df["ExpirationDate"] != nextMonthlyExp]
    totalGammaExFri.append(exFri["callGammaEx"].sum() - exFri["putGammaEx"].sum())

totalGamma = np.array(totalGamma) / 10**9
totalGammaExNext = np.array(totalGammaExNext) / 10**9
totalGammaExFri = np.array(totalGammaExFri) / 10**9

# Find Gamma Flip Point
zeroCrossIdx = np.where(np.diff(np.sign(totalGamma)))[0]

negGamma = totalGamma[zeroCrossIdx]
posGamma = totalGamma[zeroCrossIdx + 1]
negStrike = levels[zeroCrossIdx]
posStrike = levels[zeroCrossIdx + 1]

zeroGamma = posStrike - ((posStrike - negStrike) * posGamma / (posGamma - negGamma))
zeroGamma = zeroGamma[0]

In [ ]:
# Chart 3: Gamma Exposure Profile
fig, ax = plt.subplots()
plt.grid()
plt.plot(levels, totalGamma, label="All Expiries")
plt.plot(levels, totalGammaExNext, label="Ex-Next Expiry")
plt.plot(levels, totalGammaExFri, label="Ex-Next Monthly Expiry")
chartTitle = "Gamma Exposure Profile, SPX, " + todayDate.strftime("%d %b %Y")
plt.title(chartTitle, fontweight="bold", fontsize=15)
plt.xlabel("Index Price", fontweight="bold")
plt.ylabel("Gamma Exposure ($ billions/1% move)", fontweight="bold", fontsize=8)
plt.axvline(
    x=spotPrice, color="r", lw=1, label="SPX Spot: " + str("{:,.0f}".format(spotPrice))
)
plt.axvline(
    x=zeroGamma,
    color="g",
    lw=1,
    label="Gamma Flip: " + str("{:,.0f}".format(zeroGamma)),
)
plt.axhline(y=0, color="grey", lw=1)
plt.xlim([fromStrike, toStrike])
trans = ax.get_xaxis_transform()
plt.fill_between(
    [fromStrike, zeroGamma],
    min(totalGamma),
    max(totalGamma),
    facecolor="red",
    alpha=0.1,
    transform=trans,
)
plt.fill_between(
    [zeroGamma, toStrike],
    min(totalGamma),
    max(totalGamma),
    facecolor="green",
    alpha=0.1,
    transform=trans,
)
plt.legend()
plt.show()

# Printing some numbers
print(
    "Total Gamma Exposure - Estimate: {} Billions USD".format(
        round(df["TotalGamma"].sum(), 2)
    )
)